In [ ]:
#————————————————————

# Name: Azure OpenAI Assistant API, Data Pre-processing for RAG

# Purpose:  This notebook will use the Azure OpenAI Assistant API to conduct data pre-processing steps on the recipes CSV using for RAG.

# Company: Allgeier Schweiz AG
# Author: Nicolas Rehder (nrehder@allgeier.ch), Alex Dean (adean@allgeier.ch)
# Create for: SDSC 2024
# Date Created: 22.01.2024
# Last Updated: 19.05.2024
# Python Version: 3.10.4

#General Sources:
# https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/assistant
# https://learn.microsoft.com/en-us/azure/ai-services/openai/assistants-quickstart?tabs=command-line&pivots=programming-language-studio

#Azure Openai Usage:
# https://stackoverflow.com/questions/77986927/in-azure-openai-assistants-when-i-upload-a-file-and-save-it-where-is-that-file-s
# https://techcommunity.microsoft.com/t5/fasttrack-for-azure/strategies-for-optimizing-high-volume-token-usage-with-azure/ba-p/4007751#:~:text=Understanding%20tokens%20and%20limits%20in,generation%2C%20translation%2C%20or%20summarization.

#Additionals:
# https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models

# If necessary, download Python packages (run the below command in terminal if packages have not yet been installed)
# pip install -r C:\Python\sdsc\requirements.txt

#————————————————————

In [2]:
# Import Python packages
import os
import io
import time
from io import StringIO
import json
from dotenv import load_dotenv
from pathlib import Path
import pandas as pd
from openai import AzureOpenAI

In [3]:
# Load required variables from env file.
load_dotenv(dotenv_path=Path("C:\\Python\\azure-openai-lab\\.venv\\.env")) #Error sometimes due to \ or \\. Try one or the other. /workspaces/azure-openai-lab/.venv/.env

# Load Azure OpenAI Key and Endpoint. These values can be found within the Azure OpenAI Service resource in portal.azure.com under Keys and Endpoint
azure_oai_key = os.environ['AZURE_OPENAI_KEY']
azure_oai_endpoint = os.environ['AZURE_OPENAI_ENDPOINT']

In [4]:
# Initialize the Azure OpenAI client
client = AzureOpenAI(
    api_key = azure_oai_key,  
    api_version = "2024-02-15-preview",
    azure_endpoint = azure_oai_endpoint
    )

In [25]:
# Upload file into Azure OpenAI Service [NOT USED IN WORKSHOP]
# path_input = r"C:\Python\azure-openai-lab\data\recipes.csv" #Change path if required

# # send the csv file to the assistant purpose files
# response = client.files.create(
#   file=open(path_input, "rb"),
#   purpose="assistants"
# )
# print(response)
# file__id = response.id

FileObject(id='assistant-ZL9Wks4GaFf7Rt6qtsY1UtMx', bytes=2915453, created_at=1716299570, filename='recipes.csv', object='file', purpose='assistants', status='processed', status_details=None)


In [6]:
# Import existing uploaded file on Azure OpenAI Service
for i in client.files.list():
    if i.filename == "recipes":
        file__id = i.id
        print(i.id)

assistant-yJjDRA4iOizy89uOxKD9jUGR
assistant-ZL9Wks4GaFf7Rt6qtsY1UtMx


In [11]:
# Create data transformation instructions
instructions = '''
You are a senior data analyst who will work with data in a csv file in your files. 
You have access to a sandbox environment for writing Python code.
When the user asks you to perform your actions, use the csv file to read the data into a pandas dataframe.
Execute each of the steps listed below in your ACTIONS section.


ACTIONS:

1. Read the file data into a pandas DataFrame. 
2. Drop columns "id", "contributor_id" and "submitted".
3. Trim column "name" by removing irregular text spacing at the front or back of each value while keeping single spaces between words.
4. Filter the data where column "tags" contains the word vegan.
5. Create a new column named "dense_feature" combining the values of the columns "name", "tags", "nutrition", "ingredients" and "steps" separated by a semicolon.
6. Prepare a final data set named "recipes-preprocessed" that only has 50 rows randomly sampled from the dataframe from step 5.
7. Prepare recipes-preprocessed as csv files for download by the user. 

DO NOT:
1. Do not return any images. 
2. Do not return any other file types.
'''

In [12]:
# Create an Azure OpenAI Assistant
assistant = client.beta.assistants.create(
    name = "data analyst assistant",
    instructions = instructions,
    tools = [{"type": "code_interpreter"}],
    model = "gpt-4-1106-preview", #"gpt-4-0125-preview", #You must replace this value with the deployment name for your model.
    file_ids=[file__id]
)

# Get the file id
fileId = assistant.file_ids[0]

# Create a thread
thread = client.beta.threads.create()

In [13]:
# Initalize thread and start data transformation using the Azure OpenAI Assistant Code Interpreter
prompt = "Please execute your ACTIONS on the data stored in the csv file " + fileId

message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = prompt
)

In [14]:
# Run the Azure OpenAI Assistant
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

In [16]:
# Check status of Azure OpenAI Assistant run
while True:
    sec = 30
    # Wait for 30 seconds
    time.sleep(sec)  
    # Retrieve the run status
    run_status = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    # If run is completed, get messages
    if run_status.status == 'completed':
        messages = client.beta.threads.messages.list(
            thread_id=thread.id
        )
        # Loop through messages and print content based on role
        for msg in messages.data:
            role = msg.role
            try:
                content = msg.content[0].text.value
                print(f"{role.capitalize()}: {content}")
            except AttributeError:
                # This will execute if .text does not exist
                print(f"{role.capitalize()}: [Non-text content, possibly an image or other file type]")
        break
    elif run.status == "requires_action":
        # handle function calling and continue with the execution
        pass
    elif run.status == "expired" or run.status=="failed" or run.status=="cancelled":
        # run failed, expired, or was cancelled
        break    
    elif run.last_error != "None":
        # run failed, expired, or was cancelled
        break     
    else:
        print("In progress...")

Assistant: The data has been successfully processed according to the instructions, and the preprocessed dataset has been saved as a CSV file. You can download the file using the link below:

[Download recipes-preprocessed.csv](sandbox:/mnt/data/recipes-preprocessed.csv) 

Should you need further analysis or any adjustments, please let me know!
Assistant: It looks like the CSV file uses tabs (`\t`) as delimiters instead of commas. The error message indicates that the CSV reader expected 34 fields but saw 49 in line 3, which suggests that there may be lists or arrays of values within some columns. These lists might be enclosed within square brackets (`[]`) and include commas, which could lead the CSV parser to mistakenly interpret them as separate fields.

To properly read this CSV file, I will specify the tab delimiter and account for potentially complex structures within the fields. Let's attempt to read the file again with these considerations in mind.
Assistant: It seems like there w

In [23]:
# Functions to read csv files from Azure OpenAI Service
output_path = r"C:\\Python\\azure-openai-lab\\data\\" #r"/workspaces/azure-openai-lab/data/"

def read_and_save_file(first_file_id, file_name):    
    # its binary, so read it and then make it a file like object
    file_data = client.files.content(first_file_id)
    file_data_bytes = file_data.read()
    file_like_object = io.BytesIO(file_data_bytes)
    #now read as csv to create df
    returned_data = pd.read_csv(file_like_object)
    returned_data.to_csv(output_path + file_name, index=False)
    return returned_data
    # file = read_and_save_file(first_file_id, "analyst_output.csv")
    
def files_from_messages():
    messages = client.beta.threads.messages.list(
            thread_id=thread.id
        )
    first_thread_message = messages.data[0]  # Accessing the first ThreadMessage
    message_ids = first_thread_message.file_ids
    # Loop through each file ID and save the file with a sequential name
    for i, file_id in enumerate(message_ids):
        file_name = f"recipes-preprocessed.csv"  # Generate a sequential file name
        read_and_save_file(file_id, file_name)
        print(f'saved {file_name}')  

# Extract the file names from the response, retrieve the content and save the data as a csv file 
files_from_messages()

In [17]:
#Clean up Azure OpenAI environment
client.beta.assistants.delete(assistant.id)
client.beta.threads.delete(thread.id)
client.files.delete(messages.data[0].file_ids[0])


FileDeleted(id='assistant-8Grfdxht32ydSXcQrV1LkDds', deleted=True, object='file')